In [7]:
import pandas as pd
import numpy as np
import ast
pd.set_option('display.max_rows', None)

In [8]:
df.columns

Index(['run_id', 'run_config', 'dataset', 'model', 'execution_mode',
       'eval_metric', 'best_params', 'output_metrics', 'saved_model_path',
       'run_time', 'debug_preds', 'debug_ytrue'],
      dtype='object')

In [9]:
df = pd.read_csv(r"../output/autoint.csv")

In [10]:
# List of metric names
metric_names = ["f1","mse","accuracy","recall","precision", "auc","area_under_pr", "r2_score", "rmse"]
dataset = df.copy()
# Create columns based on metric names
# Convert string representations to dictionaries
dataset['output_metrics'] = dataset['output_metrics'].apply(ast.literal_eval)
# Create columns based on metric names
for metric in metric_names:
    dataset[metric] = dataset['output_metrics'].apply(lambda x: x.get(metric) if isinstance(x, dict) else np.nan)

In [12]:
# Specify the metric you want to maximize
metric_to_maximize = 'mse'
tomax = {"f1": True,
         "mse": False,
         "accuracy": True,
         "recall": True,
         "precision": True,
         "auc": True,
         "area_under_pr": True,
        "rmse": False,
        "r2_score": True}


# Filter out rows with NaN values in the specified metric column
filtered_df = dataset.dropna(subset=[metric_to_maximize]).loc[dataset[metric_to_maximize] != "nan"]

# Find the rows that maximize the specified metric for each dataset
if  tomax[metric_to_maximize]:
    max_rows = filtered_df.sort_values(["dataset",metric_to_maximize])
else:
    max_rows = filtered_df.sort_values(metric_to_maximize)
    
sel_cols = ['dataset', 'model', 'run_time',
       'eval_metric', 'best_params', 'output_metrics',
 'debug_preds', 'f1', 'mse', 'accuracy',
       'recall', 'precision', 'auc', 'area_under_pr', 'r2_score', 'rmse']

max_rows[sel_cols].head(50)

,dataset,model,run_time,eval_metric,best_params,output_metrics,debug_preds,f1,mse,accuracy,recall,precision,auc,area_under_pr,r2_score,rmse
5,housing,categoryembedding,655.336698,r2_score,"{'AdamW_weight_decay': 0.0025488672153250813, ...","{'mse': 0.3910003376579559, 'rmse': 0.62530019...","[1.9481786, 2.711327, 0.84492445, 2.8169293, 1...",NaN,0.391000,NaN,None,None,None,None,0.706610,0.625300
4,housing,resnet18,3833.859957,r2_score,"{'batch_size': 1540, 'learning_rate': 0.000138...","{'mse': 0.4255921008653211, 'rmse': 0.65237420...","[array([1.592662], dtype=float32), array([2.64...",NaN,0.425592,NaN,None,None,None,None,0.680654,0.652374


In [6]:
maximized_rows["best_params"].iloc[0]

"{'Adam_weight_decay': 0.0008344870704054452, 'ReduceLROnPlateau_factor': 0.39054754693784877, 'ReduceLROnPlateau_patience': 9, 'batch_size': 64, 'cat_emb_dim': 3, 'clip_value': 2, 'early_stopping_patience': 5, 'gamma': 1.1112005798198572, 'lambda_sparse': 0.0018702554034918367, 'learning_rate': 0.00023539065760010136, 'mask_type': 'entmax', 'momentum': 0.002024407113236184, 'n_d': 11, 'n_independent': 2, 'n_shared': 3, 'n_steps': 6, 'optimizer_fn': <class 'torch.optim.adam.Adam'>, 'scheduler_fn': 'ReduceLROnPlateau', 'virtual_batch_size_ratio': 0.5, 'weights': 1, 'outer_params': {'max_epochs': 1000, 'early_stopping_tolerance': 1e-06, 'hyperopt_evals': 10, 'val_size': 0.2, 'auto_lr_find': False}, 'virtual_batch_size': 32, 'optimizer_params': {'weight_decay': 0.0008344870704054452}, 'scheduler_params': {'factor': 0.39054754693784877, 'patience': 9, 'min_lr': 1e-07, 'verbose': True, 'mode': 'min'}, 'n_a': 11}"

In [17]:
maximized_rows["best_params"].iloc[1]

"{'alpha': 1, 'colsample_bytree': 0.7701468147287692, 'gamma': 4, 'lambda': 5, 'learning_rate': 0.019628488779340834, 'max_bin': 156, 'max_depth': 8, 'min_child_weight': 8, 'n_estimators': 255, 'subsample': 0.9280869759290384, 'tree_method': 'auto'}"